In [ ]:
from collections import Counter
from datetime import datetime
import requests
import time

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support.select import Select

from IPython.display import Audio
sound_file = './notification_sound.mp3'

In [ ]:
print('TEST AUDIO FILE')
Audio(sound_file,rate = 22050, autoplay=True)

In [ ]:
#to check all centers in selected department
url = "https://www.sante.fr/cf/centres-vaccination-covid.html"
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')


nodoctolib_list = []
doctolib_list=[]
maiia_list=[]
keldoc_list=[]
list_dep = ["78","92","75"] #"95","93","94","28","27"]
    

for dep in list_dep: 
    for link in soup.find("li", {"id":"dep-"+dep}).find_all('a'):
        link = link.get('href')      
        if link.startswith('https://partners.doctolib.fr') and "professionnel" not in link  :
            doctolib_list.append(link)
        elif link.startswith('https://www.maiia.com/'):
            maiia_list.append(link)
        elif link.startswith('https://vaccination-covid.keldoc.com/'):
            keldoc_list.append(link)
        else:   
            pass

In [15]:
#Personalized list (copy and paste link from : https://www.sante.fr/cf/centres-vaccination-covid.html)
doctolib_list=[
'https://partners.doctolib.fr/centre-de-sante/paris/centre-de-vaccination-covid-paris-15e?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/paris/centre-medical-institut-pasteur-cmip-vaccination-covid-19?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/paris/centre-de-vaccination-covid-centre-henry-dunant-croix-rouge-francaise?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/versailles/centre-de-vaccination-covid-de-versailles?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/montigny-le-bretonneux/centre-de-vaccination-covid-19-de-montigny-le-bretonneux?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/saint-germain-en-laye/centre-de-vaccination-covid-de-saint-germain-en-laye?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-vaccinations-internationales/boulogne-billancourt/centre-vaccination-ville-de-boulogne-billancourt?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/suresnes/centre-de-vaccination-covid-suresnes?speciality_id=5494&?pid=practice-164987&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/rueil-malmaison/centre-de-vaccination-covid-europe-rueil-malmaison?speciality_id=5494&enable_cookies_consent=1',
'https://partners.doctolib.fr/centre-de-sante/neuilly-sur-seine/centre-de-vaccination-covid-19-du-theatre-des-sablons?speciality_id=5494&enable_cookies_consent=1']
                                #https://www.maiia.com/centre-de-vaccination/92380-garches/centre-de-vaccination-garches

In [16]:
#Remove cookies
doctolib_list = [link[:-1]+'0' for link in doctolib_list]

In [17]:
print('nombre de lien doctolib', len(doctolib_list))
print('nombre de lien maiia', len(maiia_list))
print('nombre de lien keldoc', len(keldoc_list))

nombre de lien doctolib 10
nombre de lien maiia 1
nombre de lien keldoc 0


In [18]:
dict_list = {}
for i in range(len(doctolib_list)):
    centre_type = doctolib_list[i].split('/')[3]
    if centre_type in dict_list:
        dict_list[centre_type]+=1
    else:
        dict_list[centre_type]=1

print("TYPE DE CENTRE : ")
for k, v in sorted(dict_list.items()): print(k, v)

TYPE DE CENTRE : 
centre-de-sante 9
centre-de-vaccinations-internationales 1


In [19]:
#create first column dataframe
centre_list=[]
for url in doctolib_list:
    centre_vaccination = url.split("?")[0].split("/")[-1]
    if centre_vaccination.startswith("centre-de-vaccination-covid-19-de-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-19-de-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-covid-de-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-de-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-covid-19-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-19-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-covid-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-covid-")[1]
    elif centre_vaccination.startswith("centre-de-vaccination-"):
        centre_vaccination=centre_vaccination.split("centre-de-vaccination-")[1]
    elif centre_vaccination.startswith("ch-eure-seine-centre-de-vaccination-covid-"):
        centre_vaccination=centre_vaccination.split("ch-eure-seine-centre-de-vaccination-covid-")[1]
        
    centre_list.append(centre_vaccination)
df = pd.DataFrame(centre_list, columns=['centre'])

In [21]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')

driver = webdriver.Chrome(chrome_options=options)
wait_sec=0.5
url_booking = None

for trial in range(1,96):
    number_vaccination_list=[]
    list_with_issues=[]
    
    for i,url in enumerate(doctolib_list):

        try:
            driver.get(url)
            time.sleep(wait_sec)
            select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
            select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
            time.sleep(wait_sec)
            driver.find_element_by_xpath("//*[@id='booking-content']/div/div[3]/div[2]/div/div[1]/div/div/div[2]/div/div/div/button").click()
            url_booking = url
            break
        except NoSuchElementException:

            try:        
                select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
                select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
                time.sleep(wait_sec)
                answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                if answer == ['Désolé,', 'une', 'erreur', 'est', 'survenue.']:
                        num_vaccinations = '9999'
                if answer == ['Vous', 'ne', 'pouvez', 'pas', 'prendre', 'rendez-vous', 'en', 'ligne', 'avec', 'ce', 'praticien.']:
                    num_vaccinations = '9999'
                else:
                    num_vaccinations=answer[12]
                number_vaccination_list.append(num_vaccinations)
            except (IndexError, NoSuchElementException):  

                try:
                    answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                    if answer == ['Désolé,', 'une', 'erreur', 'est', 'survenue.']:
                        num_vaccinations = '9999'
                    if answer == ['Vous', 'ne', 'pouvez', 'pas', 'prendre', 'rendez-vous', 'en', 'ligne', 'avec', 'ce', 'praticien.']:
                        num_vaccinations = '9999'
                    else:
                        num_vaccinations =answer[12]
                    number_vaccination_list.append(num_vaccinations)
                except (IndexError, NoSuchElementException) :

                    try:
                        driver.find_element_by_xpath("//*[@id='booking-content']/div/div[1]/div[3]/label[2]").click()
                        select_fr = Select(driver.find_element_by_xpath("//*[@id='booking_motive']"))
                        select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Pfizer-BioNTech)')
                        time.sleep(wait_sec)
                        answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                        num_vaccinations=answer[12]
                        number_vaccination_list.append(num_vaccinations)

                    except (IndexError, NoSuchElementException) :
                       
                        try:
                            select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
                            select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Moderna)')
                            time.sleep(wait_sec)
                            driver.find_element_by_xpath("//*[@id='booking-content']/div/div[3]/div[2]/div/div[1]/div/div/div[2]/div/div/div/button").click()
                            Audio(sound_file,rate = 22050, autoplay=True)
                            print(url)
                        except (IndexError, NoSuchElementException) :
                            
                            try:
                                select_fr = Select(driver.find_element_by_xpath("//select[@id='booking_motive']"))
                                select_fr.select_by_visible_text('1ère injection vaccin COVID-19 (Moderna)')
                                time.sleep(wait_sec)
                                answer = driver.find_element_by_xpath("//div[@class='dl-text dl-text-body dl-text-bold dl-text-s dl-text-gray-darker']").text.split(" ")
                                if answer == ['Désolé,', 'une', 'erreur', 'est', 'survenue.']:
                                    num_vaccinations = '9999'
                                if answer == ['Vous', 'ne', 'pouvez', 'pas', 'prendre', 'rendez-vous', 'en', 'ligne', 'avec', 'ce', 'praticien.']:
                                    num_vaccinations = '9999'
                                else:
                                    num_vaccinations=answer[12]
                                number_vaccination_list.append(num_vaccinations)
                            
                            except (IndexError, NoSuchElementException) : 
                            
                                number_vaccination_list.append("0")
                                list_with_issues.append(url)

                                pass

    if url_booking is not None:
        print('URL FOR BOOKING APPOINTMENT  : ',  url_booking)
        now = datetime.now()
        print(now)
        break

    now = datetime.now()
    current_time = now.strftime("%m/%d-%H:%M:%S")
    values_df = pd.DataFrame(number_vaccination_list, columns= [current_time])
    values_df[current_time]=pd.to_numeric(values_df[current_time].replace('tous','9999'))
    df = df.join(values_df)

    pd.set_option("display.max_rows", None, "display.max_columns", None)
    if len(df.columns) >2 :
        display(df[df.iloc[:,-2]>df.iloc[:,-1]].sort_values(current_time))

    
    print('trial n.', str(trial), 'at', str(current_time) , 'terminé. Sleep for 3 min before next round')
    
    time.sleep(180)

Audio(sound_file,rate = 22050, autoplay=True)

/home/yamhiroto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


trial n. 1 at 03/03-21:41:54 terminé. Sleep for 3 min before next round


WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.182)
